In [53]:
%reset

In [54]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from scipy import linalg
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

In [55]:
cwd = Path.cwd()
project_root = cwd.parent.parent.parent
data_path_stock = project_root / "Empirical" / "data" / "stock_data.csv"
data = pd.read_csv(data_path_stock)
data_path_bond = project_root / "Empirical" / "data" / "DTB3.csv"
bond_returns = pd.read_csv(data_path_bond)

In [56]:
data_clean = data.copy()

In [57]:
data_clean['divd'] = data_clean['divd'].fillna(0)
data_clean['total_div_dollars'] = data_clean['divd'] * data_clean['cshoc']
data_clean['market_cap'] = data_clean['prccd'] * data_clean['cshoc']
data_clean = data_clean[data_clean['prccd'].notna() & (data_clean['prccd'] > 0)]
data_clean = data_clean[data_clean['cshoc'].notna() & (data_clean['cshoc'] > 0)]
data_clean = data_clean[data_clean['market_cap'] > 0]
data_clean["return"] = (data_clean["prccd"] - data_clean["prccd"].shift(1)) / data_clean["prccd"].shift(1)
data_clean['return'] = data_clean['return'].fillna(0)

In [58]:
MAX_DAILY_RETURN = 0.50   # 50% max gain (accounts for stock splits, etc.)
MIN_DAILY_RETURN = -0.50  # -50% max loss (halving in a day)

data_clean['return_is_outlier'] = (
    (data_clean['return'] > MAX_DAILY_RETURN) | 
    (data_clean['return'] < MIN_DAILY_RETURN)
)

n_outliers = data_clean['return_is_outlier'].sum()
data_clean = data_clean[~data_clean['return_is_outlier']].copy()

In [59]:
def calculate_daily_market_stats_corrected(group):
    """Calculate market statistics with CORRECTED dividend calculation"""
    
    valid = (group['market_cap'] > 0) & (group['market_cap'].notna())
    group = group[valid]
    
    if len(group) == 0 or group['market_cap'].sum() == 0:
        return pd.Series({
            'vw_return': np.nan,
            'total_dividends_paid': 0.0,
            'avg_price': np.nan,
            'total_market_cap': np.nan,
            'n_stocks': 0
        })
    
    total_mcap = group['market_cap'].sum()
    
    # Value-weighted return
    vw_return = np.sum(group['return'] * group['market_cap']) / total_mcap
    
    # CORRECTED: Sum total dollar dividends (already multiplied by shares)
    total_div_paid = group['total_div_dollars'].sum()
    
    # Value-weighted average price
    avg_price = np.sum(group['prccd'] * group['market_cap']) / total_mcap
    
    return pd.Series({
        'vw_return': vw_return,
        'total_dividends_paid': total_div_paid,
        'avg_price': avg_price,
        'total_market_cap': total_mcap,
        'n_stocks': len(group)
    })

In [60]:
daily_market = data_clean.groupby('datadate').apply(
    calculate_daily_market_stats_corrected
).reset_index()

In [61]:
daily_market = daily_market[daily_market['n_stocks'] >= 10].copy()
daily_market = daily_market.dropna(subset=['vw_return', 'avg_price', 'total_market_cap'])

In [62]:
daily_market['datadate'] = pd.to_datetime(daily_market['datadate'], errors='coerce', dayfirst=False)
bond_returns['observation_date'] = pd.to_datetime(bond_returns['observation_date'], errors='coerce', dayfirst=False)

In [63]:
daily_market = pd.merge(
    daily_market, 
    bond_returns, 
    left_on='datadate', 
    right_on='observation_date', 
    how='inner'
)

In [64]:
daily_market['bond_return'] = (1 + daily_market['DTB3']) ** (1/252) - 1
daily_market['bond_return'] = daily_market['bond_return'].bfill()

In [65]:
annual_dividends_paid = pd.Series(daily_market['total_dividends_paid']).rolling(
    window=252,
    min_periods=126
).sum().values

In [66]:
# PD ratio
total_mcap = daily_market['total_market_cap'].values
PD = total_mcap / (annual_dividends_paid + 1e-6)

# Dividend yield
div_yield = annual_dividends_paid / total_mcap

In [67]:
DivGrowth = np.full(len(annual_dividends_paid), np.nan)

for i in range(252, len(annual_dividends_paid)):
    if annual_dividends_paid[i-252] > 0:
        DivGrowth[i] = annual_dividends_paid[i] / annual_dividends_paid[i-252]
    else:
        DivGrowth[i] = 1.0

DivGrowth = pd.Series(DivGrowth).fillna(method='ffill').fillna(method='bfill').fillna(1.0).values
DivGrowth = np.clip(DivGrowth, 0.5, 2.0)

In [ ]:
stock_returns = 1 + daily_market['vw_return'].values
bond_returns = 1 + daily_market['bond_return'].values

In [72]:
# Annual stock returns
annual_ret = (np.mean(stock_returns) ** 252 - 1) * 100
annual_vol = np.std(stock_returns) * np.sqrt(252) * 100

In [73]:
# Remove initial period with NaNs
valid_mask = (~np.isnan(PD)) & (~np.isnan(DivGrowth))
first_valid = np.argmax(valid_mask)

if first_valid > 0:
    print(f"Removing first {first_valid} observations with missing data")
    stock_returns = stock_returns[first_valid:]
    bond_returns = bond_returns[first_valid:]
    PD = PD[first_valid:]
    DivGrowth = DivGrowth[first_valid:]
    daily_market = daily_market.iloc[first_valid:].reset_index(drop=True)

In [74]:
# ============================================================================
# PLOTTING CODE
# ============================================================================

import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# Get the aligned versions for plotting
dates_plot = daily_market['datadate'].values
stock_returns_plot = stock_returns
PD_plot = PD
DivGrowth_plot = DivGrowth

# Recalculate annual_dividends_paid for the cleaned data
annual_dividends_paid_plot = pd.Series(daily_market['total_dividends_paid']).rolling(
    window=252,
    min_periods=126
).sum().values

# Recalculate other needed variables
total_mcap_plot = daily_market['total_market_cap'].values
div_yield_plot = annual_dividends_paid_plot / total_mcap_plot
print("Creating plots...")

# Convert dates to proper format 
if isinstance(dates_plot[0], str):
    dates_plot = pd.to_datetime(dates_plot)
elif not isinstance(dates_plot, pd.DatetimeIndex):
    dates_plot = pd.DatetimeIndex(dates_plot)

# Create plots
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Plot 1: Daily dividends paid
print("  Plot 1...")
axes[0, 0].plot(dates_plot, daily_market['total_dividends_paid'].values, 
                linewidth=0.5, alpha=0.7)
axes[0, 0].set_title('Daily Total Dividends Paid ($)')
axes[0, 0].set_ylabel('Dividends ($ millions)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.0f}M'))

# Plot 2: Rolling annual dividends
print("  Plot 2...")
axes[0, 1].plot(dates_plot, annual_dividends_paid_plot / 1e9)
axes[0, 1].set_title('Trailing 252-Day Total Dividends')
axes[0, 1].set_ylabel('Annual Dividends ($ Billions)')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: PD ratio over time
print("  Plot 3...")
axes[1, 0].plot(dates_plot, PD_plot, linewidth=1)
axes[1, 0].set_title('Price-to-Dividend Ratio')
axes[1, 0].set_ylabel('PD Ratio')
mean_pd = np.nanmean(PD_plot)
axes[1, 0].axhline(y=mean_pd, color='r', linestyle='--', alpha=0.5, 
                   label=f'Mean: {mean_pd:.1f}')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Dividend yield over time
print("  Plot 4...")
axes[1, 1].plot(dates_plot, div_yield_plot * 100, linewidth=1)
axes[1, 1].set_title('Dividend Yield (%)')
axes[1, 1].set_ylabel('Yield (%)')
mean_dy = np.nanmean(div_yield_plot) * 100
axes[1, 1].axhline(y=mean_dy, color='r', 
                   linestyle='--', alpha=0.5, 
                   label=f'Mean: {mean_dy:.2f}%')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Plot 5: Dividend growth over time
print("  Plot 5...")
axes[2, 0].plot(dates_plot, DivGrowth_plot, linewidth=1)
axes[2, 0].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='No growth')
mean_dg = np.nanmean(DivGrowth_plot)
axes[2, 0].axhline(y=mean_dg, color='g', linestyle='--', alpha=0.5,
                   label=f'Mean: {mean_dg:.3f}')
axes[2, 0].set_title('Dividend Growth (YoY)')
axes[2, 0].set_ylabel('Growth Rate')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# Plot 6: Stock returns over time
print("  Plot 6...")
axes[2, 1].plot(dates_plot, stock_returns_plot, linewidth=0.5, alpha=0.7)
axes[2, 1].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Zero return')
axes[2, 1].set_title('Stock Returns (Gross)')
axes[2, 1].set_ylabel('Gross Return')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)
axes[2, 1].set_ylim([0.95, 1.05])

# Format x-axes for all subplots
for ax in axes.flat:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.xaxis.set_major_locator(mdates.YearLocator(2))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()

print("Saving plot...")
plt.savefig('corrected_data_diagnostics.png', dpi=150, bbox_inches='tight')  # Lower DPI for speed
print("✅ Plot saved to 'corrected_data_diagnostics.png'")

plt.close()

print("Done!")

Creating plots...
  Plot 1...
  Plot 2...
  Plot 3...
  Plot 4...
  Plot 5...
  Plot 6...
Saving plot...
✅ Plot saved to 'corrected_data_diagnostics.png'
Done!


In [49]:
print("\n" + "="*60)
print("FINAL SUMMARY STATISTICS")
print("="*60)

print(f"\nSample Information:")
print(f"  Start date: {dates_plot[0]}")
print(f"  End date: {dates_plot[-1]}")
print(f"  Number of days: {len(dates_plot):,}")
print(f"  Number of years: {len(dates_plot)/252:.1f}")

print(f"\nStock Returns:")
print(f"  Mean (daily): {np.mean(stock_returns_plot):.6f} ({(np.mean(stock_returns_plot)-1)*100:.4f}%)")
print(f"  Std (daily): {np.std(stock_returns_plot):.6f}")
print(f"  Mean (annual): {(np.mean(stock_returns_plot)**252 - 1)*100:.2f}%")
print(f"  Vol (annual): {np.std(stock_returns_plot)*np.sqrt(252)*100:.2f}%")

print(f"\nPD Ratio:")
print(f"  Mean: {np.nanmean(PD_plot):.2f}")
print(f"  Median: {np.nanmedian(PD_plot):.2f}")
print(f"  Std: {np.nanstd(PD_plot):.2f}")
print(f"  Range: [{np.nanmin(PD_plot):.2f}, {np.nanmax(PD_plot):.2f}]")

print(f"\nDividend Yield:")
print(f"  Mean: {np.nanmean(div_yield_plot)*100:.2f}%")
print(f"  Median: {np.nanmedian(div_yield_plot)*100:.2f}%")

print(f"\nDividend Growth:")
print(f"  Mean: {np.nanmean(DivGrowth_plot):.4f} ({(np.nanmean(DivGrowth_plot)-1)*100:+.2f}%)")
print(f"  Median: {np.nanmedian(DivGrowth_plot):.4f}")
print(f"  Std: {np.nanstd(DivGrowth_plot):.4f}")

print(f"\nMarket Capitalization:")
print(f"  Mean: ${np.nanmean(total_mcap_plot)/1e9:.2f} billion")
print(f"  Median: ${np.nanmedian(total_mcap_plot)/1e9:.2f} billion")

print(f"\nAnnual Dividends:")
print(f"  Mean: ${np.nanmean(annual_dividends_paid_plot)/1e9:.2f} billion")
print(f"  Median: ${np.nanmedian(annual_dividends_paid_plot)/1e9:.2f} billion")


FINAL SUMMARY STATISTICS

Sample Information:
  Start date: 2010-07-02 00:00:00
  End date: 2025-10-09 00:00:00
  Number of days: 3,842
  Number of years: 15.2

Stock Returns:
  Mean (daily): 1.000803 (0.0803%)
  Std (daily): 0.010524
  Mean (annual): 22.41%
  Vol (annual): 16.71%

PD Ratio:
  Mean: 56.13
  Median: 52.31
  Std: 9.31
  Range: [36.74, 92.25]

Dividend Yield:
  Mean: 1.82%
  Median: 1.92%

Dividend Growth:
  Mean: 1.0933 (+9.33%)
  Median: 1.0765
  Std: 0.1120

Market Capitalization:
  Mean: $23172.86 billion
  Median: $21265.68 billion

Annual Dividends:
  Mean: $410.28 billion
  Median: $420.10 billion


Now calculate the moments matrix    

In [75]:
# ============================================================================
# COMPUTE MOMENTS AND WEIGHTING MATRIX FOR DAILY DATA
# Adapted from Adam, Marcet, Nicolini (2016) computeWeightMatrix.m
# ============================================================================

import numpy as np
import pandas as pd
from scipy import linalg

print("="*60)
print("COMPUTING MOMENTS AND WEIGHTING MATRIX (DAILY DATA)")
print("="*60)

# ============================================================================
# PARAMETERS
# ============================================================================

# For daily data, adjust the lag parameter
# Original uses 5 years = 20 quarters
# For daily: 5 years = 5 * 252 = 1260 trading days
lag_years = 1
lag_days = lag_years   # 1260 trading days

method = 1  # 1 for Newey-West, 2 for den Haan-Levin
do_fullsample = 0  # 0 to use same sample for all statistics

print(f"\nParameters:")
print(f"  Lag (years): {lag_years}")
print(f"  Lag (days): {lag_days}")
print(f"  Method: {'Newey-West' if method == 1 else 'den Haan-Levin'}")

# ============================================================================
# STEP 1: Use the cleaned data from previous steps
# ============================================================================

print("\n" + "="*60)
print("USING PREPARED DATA")
print("="*60)

# These should already exist from your data preparation:
# - stock_returns (gross returns)
# - PD (price-dividend ratio)
# - DivGrowth (dividend growth)

rs = stock_returns  # Gross stock returns
PD_array = PD       # Price-dividend ratio
DivGrowth_array = DivGrowth  # Dividend growth
rb = bond_returns

print(f"\nData loaded:")
print(f"  Stock returns (rs): {len(rs):,} observations")
print(f"  Bond returns (rb): {len(rb):,} observations")
print(f"  PD ratio: {len(PD_array):,} observations")
print(f"  Dividend growth: {len(DivGrowth_array):,} observations")

COMPUTING MOMENTS AND WEIGHTING MATRIX (DAILY DATA)

Parameters:
  Lag (years): 1
  Lag (days): 1
  Method: Newey-West

USING PREPARED DATA

Data loaded:
  Stock returns (rs): 3,842 observations
  Bond returns (rb): 3,842 observations
  PD ratio: 3,842 observations
  Dividend growth: 3,842 observations


In [ ]:
print("\n" + "="*60)
print("BUILDING ADDITIONAL SERIES")
print("="*60)

# -------------------------------
# PD(t) * PD(t-1) (For autocorrelation of PD Ratio)
# -------------------------------
PD_array = np.asarray(PD_array)  # ensure numpy array
PD_PDlag = np.full(len(PD_array), np.nan)
PD_PDlag[1:] = PD_array[1:] * PD_array[:-1]

# -------------------------------
# rs(t) * rs(t-1) (For autocorrelation of Returns)
# -------------------------------
rs = np.asarray(rs)
rs_rslag = np.full(len(rs), np.nan)
rs_rslag[1:] = rs[1:] * rs[:-1]

# -------------------------------
# Daily excess returns
# -------------------------------
# Ensure rb is numpy array
rb = np.asarray(rb)

# excess return
excret = rs - rb

# Convert to pandas Series 
excret = pd.Series(excret, name="excess_return")

# -------------------------------
# PD * daily excess return
# -------------------------------
# Align lengths: PD_array and excret
PD_excret = PD_array * excret
PD_excret = pd.Series(PD_excret, name="PD_excess_return")



BUILDING ADDITIONAL SERIES
  ✅ PD_PDlag length: 3842
  ✅ rs_rslag length: 3842
  ✅ Daily excess return length: 3842
  ✅ PD * daily excess return length: 3842
  ✅ All series built successfully


In [78]:
# ============================================================================
# STEP 6: Build h(y) matrix
# ============================================================================
# h(y) is the moments vecotr at the observational level
# the result is a 13xN matrix where each row corresponds to a specific empirical moment function based on the observed data

n_obs = len(rs) - 1 - lag_days
print(f"  Observations: {n_obs}")

# Initialize
hy = np.zeros((13, n_obs))

# Fill matrix
hy[0, :] = rs[1:-lag_days]
hy[1, :] = rb[1:-lag_days]
hy[2, :] = PD_array[1:-lag_days]
hy[3, :] = DivGrowth_array[1:-lag_days]
hy[4, :] = hy[0, :] ** 2
hy[5, :] = hy[2, :] ** 2
hy[6, :] = hy[3, :] ** 2
hy[7, :] = PD_PDlag[1:-lag_days]
hy[8, :] = excret[1:-lag_days]
hy[9, :] = excret[1:-lag_days] ** 2
hy[10, :] = PD_excret[1:-lag_days]
hy[11, :] = hy[1, :] ** 2
hy[12, :] = rs_rslag[1:-lag_days]

print(f"  Matrix shape: {hy.shape}")

N = hy.shape[1]
M = np.mean(hy, axis=1)


  Observations: 3840
  Matrix shape: (13, 3840)


In [ ]:
# ============================================================================
# STEP 7: Compute statistics S(M)
# ============================================================================
# this cell maps the observed moments into statistics (some functions S are nonlinear, e.g.: R2, autocorellation)

print("\n" + "="*60)
print("COMPUTING STATISTICS")
print("="*60)

S = np.zeros(12)

S[0] = M[0]
S[1] = M[1]
S[2] = M[2]
S[3] = M[3]
S[4] = np.sqrt(max(M[4] - M[0]**2, 1e-10))
S[5] = np.sqrt(max(M[5] - M[2]**2, 1e-10))
S[6] = np.sqrt(max(M[6] - M[3]**2, 1e-10))
S[7] = (M[7] - M[2]**2) / (S[5]**2 + 1e-10)
S[8] = (M[10] - M[2]*M[8]) / (S[5]**2 + 1e-10)
S[9] = S[8]**2 * S[5]**2 / max(M[9] - M[8]**2, 1e-10)
S[10] = np.sqrt(max(M[11] - M[1]**2, 1e-10))
S[11] = (M[12] - M[0]**2) / (S[4]**2 + 1e-10)

stat_labels = [
    'E(r^s)', 'E(r^b)', 'E(PD)', 'E(D/d)',
    'sigma_r^s', 'sigma_PD', 'sigma_D/d', 'autcorr_PD',
    'betaPD', 'R^2', 'sigma_r^b', 'autcorr_r^s'
]

print("\n" + "="*60)
print("EMPIRICAL MOMENTS (DAILY DATA)")
print("="*60)
for i, (label, value) in enumerate(zip(stat_labels, S)):
    print(f"{i+1:2d}. {label:<15}: {value:10.6f}")

# Annualized
annual_ret = (S[0] ** 252 - 1) * 100
annual_vol = S[4] * np.sqrt(252) * 100

print(f"\nAnnualized:")
print(f"  Return: {annual_ret:.2f}%")
print(f"  Volatility: {annual_vol:.2f}%")



COMPUTING STATISTICS

EMPIRICAL MOMENTS (DAILY DATA)
 1. E(r^s)         :   1.000805
 2. E(r^b)         :   1.002490
 3. E(PD)          :  56.115434
 4. E(D/d)         :   1.093333
 5. sigma_r^s      :   0.010527
 6. sigma_PD       :   9.291257
 7. sigma_D/d      :   0.111995
 8. autcorr_PD     :   1.000157
 9. betaPD         :  -0.000063
10. R^2            :   0.002863
11. sigma_r^b      :   0.002670
12. autcorr_r^s    :  -0.144499

Annualized:
  Return: 22.48%
  Volatility: 16.71%


In [80]:
import numpy as np

print("\n" + "="*60)
print("COMPUTING JACOBIAN (VECTORIZED & SAFE)")
print("="*60)

# Small epsilon to avoid divide-by-zero
eps = 1e-10

# Ensure S elements that appear in denominators are safe
S_safe = np.copy(S)
S_safe[[4,5,6,10]] = np.maximum(S_safe[[4,5,6,10]], eps)

# Initialize Jacobian
dS = np.zeros((len(S), len(M)))

# -------------------------------
# 1. Means are direct derivatives
# -------------------------------
for i in range(4):
    dS[i, i] = 1.0

# -------------------------------
# 2. Standard deviations: sigma = sqrt(M_var - M_mean^2)
# -------------------------------
dS[4, 4] = 1.0 / (2*S_safe[4])
dS[4, 0] = -M[0] / S_safe[4]

dS[5, 5] = 1.0 / (2*S_safe[5])
dS[5, 2] = -M[2] / S_safe[5]

dS[6, 6] = 1.0 / (2*S_safe[6])
dS[6, 3] = -M[3] / S_safe[6]

# -------------------------------
# 3. Autocorrelation: rho = (M_lag - M_mean^2) / sigma^2
# -------------------------------
dS[7, 7] = 1.0 / S_safe[5]**2
dS[7, 2] = -2*M[2] / S_safe[5]**2

# -------------------------------
# 4. Beta (S[8]) and R² (S[9])
# -------------------------------
# Beta: beta = (M[10] - M[2]*M[8]) / sigma_PD^2
dS[8, 2] = (-M[8]*S_safe[5]**2 - 2*S_safe[5]*(-M[2]/2))*S_safe[5]**-2
dS[8, 5] = -2*(S[8]/S_safe[5])*dS[5,5]  # chain rule
dS[8, 8] = -M[2]/S_safe[5]**2
dS[8,10] = 1.0 / S_safe[5]**2

# R²: R² = beta^2 * sigma_PD^2 / (var(Y))
denom_r2 = M[9] - M[8]**2
denom_r2 = max(denom_r2, eps)
dS[9,2]  = (2*S[8]*dS[8,2]*S_safe[5]**2 + 2*S[8]**2*S_safe[5]*dS[5,2])/denom_r2
dS[9,5]  = (2*S[8]*dS[8,5]*S_safe[5]**2 + 2*S[8]**2*S_safe[5]*dS[5,5])/denom_r2
dS[9,8]  = ((2*S[8]*dS[8,8]*S_safe[5]**2 + 2*S[8]**2*S_safe[5]*0)*denom_r2 - (-2*M[8]*S[8]**2*S_safe[5]**2))/denom_r2**2
dS[9,9]  = -S[8]**2*S_safe[5]**2 / denom_r2**2
dS[9,10] = 2*S[8]/denom_r2

# -------------------------------
# 5. Remaining standard deviations and autocorrelation
# -------------------------------
dS[10,11] = 1.0 / (2*S_safe[10])
dS[10,1]  = -M[1]/S_safe[10]

dS[11,12] = 1.0 / S_safe[4]**2
dS[11,0]  = 2*M[0]*(M[12]-M[4])/S_safe[4]**4
dS[11,4]  = -2*(S[11]/S_safe[4])*dS[4,4]

print(f"  Shape: {dS.shape}, Non-zero elements: {np.sum(dS != 0)}")



COMPUTING JACOBIAN (VECTORIZED & SAFE)
  Shape: (12, 13), Non-zero elements: 26


In [ ]:
# ============================================================================
# STEP 9: Newey-West
# ============================================================================

print("\n" + "="*60)
print("COMPUTING NEWEY-WEST")
print("="*60)

Mn = np.tile(M.reshape(-1, 1), (1, N))
hy_Mn = hy - Mn
C0 = (1.0 / N) * (hy_Mn @ hy_Mn.T)

K = int(np.floor(N / 4))
print(f"  K = {K}")

Cj = np.zeros((len(M), len(M), K))
for j in range(K):
    wjK = (1 - (j + 1) / (K + 1)) * (1.0 / (N - j - 1))
    Cj[:, :, j] = wjK * (hy_Mn[:, j+1:] @ hy_Mn[:, :-(j+1)].T)

Sw = C0.copy()
for j in range(K):
    Sw = Sw + (Cj[:, :, j] + Cj[:, :, j].T)

print(f"  ✅ Sw computed")


COMPUTING NEWEY-WEST
  K = 960
  ✅ Sw computed


In [ ]:
# ============================================================================
# STEP 10: Weight matrix
# ============================================================================

print("\n" + "="*60)
print("WEIGHT MATRIX")
print("="*60)

dS_Sw_dS = dS @ Sw @ dS.T

eigenvalues = linalg.eigvals(dS_Sw_dS)
if np.min(eigenvalues.real) > 0:
    print("  ✅ Positive definite")
    W = linalg.inv(dS_Sw_dS)
else:
    print("  ⚠️  Using pseudo-inverse")
    W = linalg.pinv(dS_Sw_dS)


WEIGHT MATRIX
  ⚠️  Using pseudo-inverse


In [ ]:
# ============================================================================
# SAVE
# ============================================================================

np.savez('daily_moment_calculations.npz',
         S=S, dS_Sw_dS=dS_Sw_dS, W=W, M=M, dS=dS, Sw=Sw,
         N=N, lag_days=lag_days,
         stat_labels=np.array(stat_labels, dtype=object))

pd.DataFrame({'Statistic': stat_labels, 'Value': S}).to_csv('daily_empirical_moments.csv', index=False)

print("\n✅ COMPLETE! Results saved.")


✅ COMPLETE! Results saved.
